In [ ]:
# 라이브러리 import 및 logging 설정
import requests
from bs4 import BeautifulSoup
import time
import logging

import data_utils

data_utils.setup_logging()
logging.info("필수 라이브러리 및 data_utils 모듈 임포트, 로깅 설정 완료")

In [ ]:
# 직종 구분 선택 -> 사이트마다 다르게 설정할 것
job_category = "total" # "total" # "backend" # "frontend"

if job_category == "total":
    job_category_url = "?jobGroup=DEVELOPER"
elif job_category == "backend":
    job_category_url = "?job=BACKEND_DEVELOPER&jobGroup=DEVELOPER"
elif job_category == "frontend":
    job_category_url = "?job=FRONTEND_DEVELOPER&jobGroup=DEVELOPER"
else:
    job_category == "total"
    job_category_url = "?jobGroup=DEVELOPER"

In [ ]:
# 스크래핑 설정
scraped_data = []
start_page_no = 1

# 아래 사이트 관련은 사이트 별로 변경 필수
base_url = "https://www.rallit.com"
site_name = 'rallit'

headers = data_utils.DEFAULT_HEADERS

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("스크래핑 설정 완료.")

In [ ]:
# CSS 선택자를 이용하여 soup 객체에서 데이터를 추출하는 함수.
# 이 함수는 현재 스크립트(Notebook)에서 사용됩니다.
def extract_data(soup, selector, attribute=None, is_list=False, default="N/A"):
    """
    주어진 CSS 선택자를 사용하여 BeautifulSoup 객체에서 데이터를 추출합니다.
    Args:
        soup (BeautifulSoup): 데이터를 추출할 BeautifulSoup 객체.
        selector (str): CSS 선택자.
        attribute (str, optional): 추출할 속성. None이면 텍스트 추출. Defaults to None.
        is_list (bool, optional): 여러 개의 결과를 반환할지 여부. Defaults to False.
        default (str, optional): 찾지 못했을 경우 반환할 기본값. Defaults to "N/A".

    Returns:
        str or list: 추출된 텍스트 또는 속성 값 (is_list=True인 경우 리스트).
    """
    try:
        if is_list:
            elements = soup.select(selector)
            if attribute:
                return [element.get(attribute, default).strip() for element in elements]
            else:
                return [element.get_text(strip=True) for element in elements]
        else:
            element = soup.select_one(selector)
            if element:
                if attribute:
                    return element.get(attribute, default).strip()
                else:
                    return element.get_text(strip=True)
            else:
                # print(f"Warning: Could not find element with selector '{selector}'.") # 로깅으로 대체 권장
                logging.warning(f"Could not find element with selector '{selector}'.")
                return default
    except Exception as e:
        # print(f"Error extracting data with selector '{selector}': {e}") # 로깅으로 대체 권장
        logging.error(f"Error extracting data with selector '{selector}': {e}", exc_info=True)
        return default

In [ ]:
# 실제 스크래핑 루프 실행
page_no = start_page_no
while True: # 무한 루프 시작 (종료 조건은 루프 내에 있음)
    # 대상 URL 구성
    list_url = f"{base_url}{job_category_url}&pageNumber={page_no}"
    logging.info(f"Scraping page: {list_url}")

    try:
        # 1. 채용 공고 목록 페이지 가져오기
        response = requests.get(list_url, headers=headers)
        response.raise_for_status() # HTTP 오류 발생 시 예외 발생
        soup = BeautifulSoup(response.text, 'html.parser')

        # 2. 각 채용 공고 컨테이너(article) 찾기
        job_postings = soup.find_all('article')

        # *** 동적 로딩 확인 및 루프 종료 조건 ***
        # 페이지에 공고가 없거나, 개발자 공고 제목 요소를 찾지 못하면 종료
        if not job_postings or not soup.select_one('h3.summary__title.css-5g43jj'):
            if not job_postings:
                logging.info(f"Page {page_no}: 채용 공고 article 태그를 찾을 수 없습니다. 스크래핑을 종료합니다.")
                print("\n채용 공고 article 태그를 찾을 수 없습니다. 스크래핑을 종료합니다.")
            else: # job_postings는 있지만, 특정 css selector가 없는 경우
                logging.info(f"Page {page_no}: 더 이상 개발자 공고 제목(h3.summary__title.css-5g43jj)을 찾을 수 없습니다. 스크래핑을 종료합니다.")
                print("\n더 이상 개발자 공고 제목을 찾을 수 없습니다. 스크래핑을 종료합니다.")
            break # 공고가 없거나 종료 조건 만족 시 루프 종료
        else:
            logging.info(f"Found {len(job_postings)} job postings on page {page_no}.")
            print(f"Found {len(job_postings)} job postings on page {page_no}.")


        # 3. 각 공고별 정보 추출
        for item in job_postings:
            company_name = "N/A"
            position = "N/A"
            skill = "N/A"

            # 각 채용 공고 article 안의 div.css-vjt50z 컨테이너 찾기
            job_info_container = item.find('div', class_='css-vjt50z')

            if job_info_container:
                # 3-1. 회사명 추출
                company_name_element = job_info_container.find('p', class_='summary__company-name css-x5ccem')
                if company_name_element:
                    company_name = company_name_element.get_text(strip=True)

                # 3-2. 공고 제목 추출
                position_element = job_info_container.find('h3', class_='summary__title css-5g43jj')
                if position_element:
                    position = position_element.get_text(strip=True)

                # 3-3. 상세 페이지 링크 추출 (모든 skill 추출)
                skill_elements = job_info_container.find_all('p', class_='css-13kyeyo')
                skills_list = [skill_el.get_text(strip=True) for skill_el in skill_elements] # 변수명 충돌 피함
                skill = ', '.join(skills_list) if skills_list else "N/A"

                # --- 여기에 filtering_utils 모듈의 필터링 함수 적용 ---
                filtered_skill = data_utils.filter_skill_data(skill)
                # --- 필터링 완료 ---

                # 3-5. 결과 저장
                scraped_data.append({
                    'company': company_name,
                    'position': position,
                    'skill': filtered_skill,
                })
                # 상세 로그는 너무 길 수 있으니 필요에 따라 주석 처리하거나 레벨 조정
                # logging.debug("    --- Scraped Data ---")
                # logging.debug(f"    company: {company_name}")
                # logging.debug(f"    position: {position}")
                # logging.debug(f"    skill: {filtered_skill[:100]}..." if filtered_skill and len(filtered_skill) > 100 else filtered_skill if filtered_skill else "N/A")
                # logging.debug("    ----------------------")
            else:
                logging.warning("    --- No job info container found for an article element ---")
                # print("    --- No job info container found ---") # 로깅으로 대체 권장

        page_no += 1 # 다음 페이지로 이동
        print(f"Current page number: {page_no}")
        time.sleep(1) # 페이지 간 딜레이 (필수)

    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching list page {list_url}: {e}", exc_info=True)
        print(f"\nError fetching list page {list_url}: {e}")
        break # 네트워크 또는 HTTP 오류 발생 시 루프 종료
    except Exception as e:
        logging.error(f"An unexpected error occurred during scraping: {e}", exc_info=True)
        print(f"\nAn unexpected error occurred: {e}")
        break

logging.info("스크래핑 루프 종료.")

In [ ]:
# data_utils 모듈의 save_data_to_csv 함수를 사용하여 데이터를 저장합니다.
data_folder = 'data'
filename = f'data_{site_name}_{job_category}.csv'

saved_filepath = data_utils.save_data_to_csv(scraped_data, filename, folder=data_folder)

In [ ]:
# 저장된 CSV 파일을 읽어와 DataFrame으로 출력 (확인용)
data_utils.load_data_from_csv(saved_filepath)